# Cosmic Ray Counting - Individual Analysis

A big part of this analysis includes a numerical calculation of the acceptance of the muon counting paddle array.  You will want to discuss your approach with the instructor.  The calculation will make use of some numerical integration methods.

You may also use the [Uncertainties](https://pythonhosted.org/uncertainties/) package for calculating statistical uncertainty. 

Consult the documentation for different Python packages.  Also recommended: the [Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/) and the [Python Data Science Handbook] (https://jakevdp.github.io/PythonDataScienceHandbook/) both by Jake VanderPlas.

In [ ]:
import numpy as np  # import the numpy library functions.
import scipy.constants as const # import physical constants
import matplotlib.pyplot as plt # plotting functions
import uncertainties as unc # Uncertainties package.  Good for simple error propagation
# directive below puts the plots in the notebook
%matplotlib inline

## Outline

### Acceptance calculations
1. Calculate the estimated acceptance of the telescope based on the suggested approximation and the measurements taken.  Copy the result to your group notebook and give a brief discussion of whether this estimate is likely to be too low or too high there.
2. Derive the expected “singles” rate for one detector paddle (Exercise 1). Include the result in your group notebook.
3. Use numerical integration to eliminate simplifying assumption 2: calculate the flux $J$ through a rectangular-shaped solid angle defined by the rectangle of the upper paddle and an element $dA$ at the center of the lower rectangular paddle.  Compare this calculation with the simplified one.
4. **OPTIONAL:** extend the numerical integration to sum over different elements $dA$ of the lower paddle.

### Detector efficiency calculations
1. Include your calculations to determine the counting efficiency of each paddle in the array.
2. Calculate the expected rates of 2-fold and 3-fold accidental coincidences.  How long would you need to count in order to test these estimates to within 10% uncertainty?

### Final results
1. Calculate the muon flux from the data set, and compare it to the expected value.
2. Correct your results by using the estimated efficiency of the paddle detectors.  
3. **OPTIONAL:** From long counting runs with two paddles close together, for example (A & B), and two paddles far apart, for example (A & D), calculate the muon flux and compare with expectations.



## Acceptance

### Create functions for solid angle and estimated flux
Create two functions that calculate the results of equations (1) and (2) in the instructions.  Equation (1) gives the solid angle subtended by the upper paddle as seen from a small element of area at the center of the lower paddle:
$$\Omega(\theta) = \int_\text{cone}d\Omega = 2\pi\left(1-\cos\theta\right)\;.\quad\quad\quad\quad(1)$$
Equation (2) gives the estimated flux through the same element of area based on the approximate angular distribution of flux $I(\theta) = I_v\cos^2\theta$:
$$J(\theta) = \int_\text{cone}I(\theta')\cos\theta'\,d\Omega=\left(\frac{\pi}{2}\right)I_v\left(1-\cos^4\theta\right)\;.\quad\quad\quad\quad(2)$$

The angle $\theta$ is given by the angle between the center of the paddle and the outer rim located at a radius $R$ relative to the center of the lower paddle whose separation is $h$.  Thus $\cos\theta = h/\sqrt{R^2+h^2}$.  The radius $R$ is chosen so that the area of the circular paddle would be the same as the area of the actual rectangular paddle.

Your functions should have as arguments the `width` and `length` of the paddle, and `separation` between the top and bottom paddles in your setup.  (Note: use consistent units for these; centimeters or inches are OK.)

In [ ]:
# Here is a function template:

def conical_Omega(width,length,separation):
    '''
    Solid angle subtended by a cone defined by the area of the width x length 
    and the separation.
    '''
    # You fill in function

    
def conical_Flux(width,length,separation):
    '''
    Normalized flux through the cone and unit area at center of bottom paddle
    '''
    # You fill in function
    
# Test your functions below.  For separation of zero, Omega should be 2*pi and 
# the normalized flux should be pi/2 (times I_v) 

sep = 0.0
# Note: I'm using inches for my padle dimensions
print('Solid angle for {:.1f} circular paddle separation = {:.4f} sr'.format(sep,conical_Omega(6.0, 12.0, sep)))
print('Flux for {:.1f} circular paddle separation = {:.4f} I_v'.format(sep,conical_Flux(6.0, 12.0, sep)))


### Predicted singles rate

For a separation of zero, verify that the expected flux would be 0.0127 particles per cm<sup>2</sup> per second given the value of $I_v$ stated in the instructions.  Then multiply this by the area of a paddle to estimate the expected "true" singles rate in counts per second and counts per minute.

In [ ]:
# You write this



### Eliminate the circular paddle assumption

We will solve this problem in steps.  This will help reduce the likelihood of a calculation mistake.

#### Step 1: Learn how to calculate by integration the solid angle subtended by the face of a cube
Based on the following facts, calculate the solid angle seen from the center of a cube out through one face by direct integration.  The answer should be $4\pi/6$ because a full cube would see the same solid angle as a sphere, and a cube has 6 equal faces.

The facts are

1. The differential unit of solid angle subtended by a small patch of surface $dA$ located a distance $r$ from an origin is given by the relationship $$d\Omega = \frac{\hat{n}\cdot d\vec{A}}{r^2}$$ where $\hat{n}$ is the unit radial vector from the origin and $d\vec{A}$ is the differential area whose direction is perpendicular to the patch.
2. For a patch lying parallel to the $x,y$ plane (i.e., $d\vec{A}$ points along the $z$ axis) in rectagular coordinates, $\hat{n}\cdot d\vec{A} = \cos\theta\,dx\,dy$ where $\theta$ is the angle between the $z$ axis and $r$, the magnitude of $d\vec{A}$ equals $dx\,dy$, and $$\cos\theta = \frac{z}{r} = \frac{z}{\sqrt{x^2+y^2+z^2}}$$.
3. If the cube has sides of length $2a$ then $z=a$ and the solid angle subtended by one of the faces is given by the double integral $$\Omega(A) = \int_A d\Omega = \int_{-a}^{a}\int_{-a}^{a}\frac{a\,dx\,dy}{(x^2+y^2+a^2)^{3/2}}\;.$$  The value of this integral should be $2\pi/3$.

#### Step 2: Apply SciPy's `dblquad()` function

There are two parts to accomplish to use `dblquad()`, which allows one to integrate a two dimensional function over a surface: (1) Create a function to integrate&mdash;the integrand.  It has to follow a standard template. (2) Feed the into `dblquad()`, making sure to handle the integration limits and any parameters passed to the function correctly.

##### Create the integrand
The first two variables should be the inner integration variable and outer integration variable.  For example in the above, this would be $x$ and $y$.  Other variables in the function come after that (such as the separation for our paddles, or in this case, the distance between the center of the cube and the face).

Start by writing and testing a function that gives the integrand in item 3 above.

In [ ]:
# Complete the function below that gives d-Omega
def delta_Omega(x,y,a):
    '''
    Integrand to calculate solid angle subtended by a 
    rectangular sheet at a distance a from the origin.
    '''


##### Import the integration routines
Next, import from SciPy the integration routines.  (Just execute this cell.)

In [ ]:
import scipy.integrate as integrate

Read about `dblquad()` in the [SciPy **integrate** documentation](https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html)

In particular, pay attention to the order of the variables.  The inner limits can be functions of the outer variables.  In our case, we do not need to define such functions, but instead use numbers (i.e., "floats") for both sets of limits.

You will also need to pass the value of the separation, which does not change in the integration, as an additional parameter.  This must be placed into a "tuple" even if it is a single quantity and then assigned to *keyword variable* `args`.

Here is an example of a call that would use `delta_Omega()` as defined above:

    # x limits
    xlow = -1.0
    xhi = +1.0
    # y limits
    ylow = -2.0
    yhi = +2.0
    # separation
    sep = 3.0
    
    # Call to dblquad().  The return is a tuple with the first element being the value of 
    # the integral.  The index at the end selects this element.
    omega = integrate.dblquad(delta_Omega, xlow, xhi, ylow, yhi, args=(sep,))[0]

Write code similar to the above and test your integration.   If all the integration limits have the same magnitude equal to the separation parameter `sep`, this corresponds to the cube scenario described above and the result should be $2\pi/3$. If `sep` is close to zero, the result should be close to $2\pi$.   (What happens if `sep` is set exactly zero?)

In [ ]:
# Write code to test the integration call.  



##### Vectorize it!
To create results for a range of separations, it is helpful to wrap the integration into a function call and then "vectorize" that calling function.  This makes it play well with numpy arrays, and also speeds up calculation.  (You don't need a "for loop"!)

Here is what this would look like, given the above. Define a wrapper function (and simplfy the calling variables:
    
    def Omega(width,length,sep):
        return integrate.dblquad(delta_Omega, -width/2, width/2, -length/2, length/2, args=(sep,))[0]

"Vectorize" it:

    vec_Omega = np.vectorize(Omega)
    
Then you can run the integration over a range of separation values more quickly and without using a for loop:

    separations = np.arange(0.01,22.0,0.2) # units = inches.  First value is close to zero.
    omegas = vec_Omega(6.0, 12.0, separations)
    
Without vectorizing, to do the above would require something like this, and it would also take longer:

    separations = np.arange(0.01,22.0,0.2)
    omegas = np.zeros(0)
    for separation in separations:
        this_omega = Omega(6.0, 12.0, separation)
        omegas = np.append(omegas, this_omega)

In [ ]:
# Wrap your integration into a function, vectorize it, and test it by creating an 
# array of solid angles given an array of separations 


#### Step 3: Plot the results for solid angle

Compare the solid-angle numerical integration to the result from the conical model (equation 1 above) by plotting both calculations on the same graph.

In [ ]:
# Create the arrays for the conical model



In [ ]:
# Plot the results on a large graph.  The first lines do this.

plt.figure(figsize=(14,10))
plt.grid()
# You complete the rest



#### Step 4: Repeat with flux calculation

The *downward* flux density from a region subtended by solid angle $\Omega$ is given by the integral $$ J = \int_\Omega I(\theta')\cos\theta'\,d\Omega\,, $$ which for a cone defined by the apex angle $\theta$ results in equation (2) above.  $I(\theta)$ is the flux density per steradian, and it also has an angular depepndence.  The factor of $\cos\theta'$ in the integral accounts for the fact that flux coming in at an angle will have a lower *downward* value in proportion to the particle velocity in the $z$ direction.

To calculate $J$ for our rectangular paddle, we need to cast $I(\theta)\cos\theta$ into rectangular coordinates.  This is easy because we can assume $I(\theta) = I_v\cos^2\theta$ where $I_v$ is a constant and we know that $\cos\theta = a/\sqrt{x^2+y^2+a^2}$.

Thus, to find a model for the flux, you only need to create a new integrand that incorporates $I(\theta)\cos\theta$ along with the differential solid angle, and the rerun the integration steps:

##### 1. Write a new integrand  

In [ ]:
# Complete the function below that gives d-J
def delta_J(x,y,a):
    '''
    Integrand to calculate solid angle subtended by a 
    rectangular sheet at a distance a from the origin.
    '''

##### 2. feed it into `dblquad()`

In [ ]:
# Copy what you did above (and test it)



##### 3. Vectorize it

And, create arrays to calculate flux as a function of separation.

In [ ]:
# Copy what you did above



##### 4. Plot it

As before, plot in comparison to the round-paddle version

In [ ]:
# You write this



### Calculate the expected count rate

Now that you have two versions of a "normalized" flux per unit area that would pass through a patch located at the center of the bottom paddle, calculate a first pass at the expected count rate.  You need to multiply your result by $I_v$ and by the area of the bottom paddle.  

Carry out these calculations below, and print your result with correct significant figures. 

In [ ]:
# Add as many cells as you need



### OPTIONAL: Full integration

Because each small area of the lower paddle does not see the same part of the sky through the upper paddle, the count rate per area of the lower paddle should not be constant.  To correct for this, one would need to adjust the limits of integration depending on where the sight line comes from, and then add up the results.

To do this, think of a spot on the lower paddle, as measured from an origin located at the paddle center.  This coordinate could also mark a spot on the upper paddle, and set the point from which the integration limits are marked.  For example, if the coordinate is $(X,Y)$, and we denote the paddle width by $W$ and length by $L$, then the $x$-integration limits would change from $-W/2$ & $W/2$ to $X-W/2$ & $X+W/2$. Likewise, along the other dimension, we would get limits of $Y-L/2$ & $Y+L/2$.  

One divides up a paddle into small areas and creates an array of positions to calculate the integrals from.  Then it is a matter of summing the results and scaling by the area of the small patches.  By symmetry, one only needs to operate on one quadrant of the lower paddle, and then multiply the result by 4.

If you are up for it, give it a try, and plot your result on top of the ones calculated using the approximations.

## Paddle Efficiencies & Accidentals

### Paddle efficiencies

You made sets of measurements to determine the efficiency of each paddle.  These calculations are based on the model that a particle which passes through the array will get recorded according to the product of the efficiency of each paddle in the array, assuming a simultaneouls pulse from all paddles is required to record a pulse.  The combined efficiency of a three-paddle array is, for example, $E_{ABD} = E_AE_BE_D$.  Likewise the efficiency of a two paddle array is $E_{AD}=E_AE_D$.  Thus in a simultaneous measurement of counts for the two different configurations, we expect

$$ E_B = \frac{E_{ABD}}{E_{AD}} = \frac{N_{ABD}}{N_{AD}}\;. $$

Use as many cells as you need to combine your measurements from 2-, 3-, and 4-fold counting to derive estimates of the efficiencies of all of the paddles in your array.  

The output of the last cell should summarize the best-known efficiency of each paddle.

In [ ]:
# Add as many cell here as you need



### Accidentals  rates

In the cell below, define variables and calculate with them to estimate the rates of 2-fold and 3-fold accidental coincidences, given your setup and measured singles rates.

Your rates should be given in terms of counts/second and in terms of the average interval time between counts (seconds).  If the average interval is very long, convert your units to hours, days, or years as appropriate to human understanding.

How long would you need to wait to get the values to within 10%, based on Poisson statistics?

In [ ]:
# Add as many cell here as you need



## Final results

First, record your data for your "long count" that gives the counts/time for your array.  Based on your run time and total counts, what would be the uncertainty in this number?  (It should be pretty small.)

Then compare this value to the expected count rate you calculated above.  

In [ ]:
# Add as many cell here as you need



### Correct your result

Include the efficiencies in the paddle array you used to adjust the counting rate upward.  Does this make the result agree with expectations?  (No worry if it is not, unless it differs by orders of magnitude, in which case you probably made a calculation mistake.)

In your group notebook discuss these results:
1. Does eliminating the circular-paddle approximation have much effect on the expected counting rate?  Why or why not?
2. Would you expect the second simplifying assumption, that the flux through the array is given simply by the flux density through the senter of the array times the area of the lower paddle to over or under estimate the expected flux? Explain.
3. What would you expect to be the effect of making the measurements inside a second-floor lab in a four-story building, compared to running the experiment outside?

In [ ]:
# Add as many cell here as you need



### OPTIONAL: Include other measurements

If you have other measurements, such as rates from two close-together paddles and from two far-apart paddles, estimate the flux from arrays with these separations, including your efficiency corrections as needed.  

It would be interesting to plot your measurements on the graph for the expected flux as a function of paddle separation.  You will need to either scale the theory by $I_v$ or calculate an experimental $I_v(\text{expt})$ to divide into your measurements for plotting.